# BIDS Converter 

## Import

In [ ]:
import matplotlib
import pathlib

import os
import mne
import mne_bids

from mne_bids import (write_raw_bids, BIDSPath,
                      make_report, print_dir_tree)
from mne_bids.stats import count_events
from pathlib import Path

matplotlib.use('Agg') #Raises no browsers
#matplotlib.use('Qt5Agg') #Raises browsers

## Print Name

In [ ]:
printname = 'sub-'+subject+'_ses-'+session+'_task-'+task

In [ ]:
print('######################## Input: ['+printname+'] ########################')

## Set Path

In [ ]:
mffname = task+'.mff'

In [ ]:
raw_path = os.path.join(root_path, "EGI_RAW/"+subject+"/"+session+"/"+mffname)

## Read Raw

Modifying data in-place
https://mne.tools/stable/auto_tutorials/intro/15_inplace.html


In [ ]:
raw = mne.io.read_raw_egi(raw_path)
raw.load_data() 

## Label Events

Parsing events from raw data
https://mne.tools/stable/auto_tutorials/intro/20_events_from_raw.html

Working with events
https://mne.tools/stable/auto_tutorials/raw/20_event_arrays.html

In [ ]:
pre_events = mne.find_events(raw, 'STI 014')
events = mne.pick_events(pre_events)#, exclude=[1, 2, 3, 4, 6])
event_id = {
    'CELL': 1,
    'bgin': 2, #when each trial starts
    'TRSP': 3, 
    'fix+': 4, #when the fixation cross appears
    'WMD+': 5, #onset of the working memory display 
    'TsD-': 6 #onset of the test display
}

events 

## Power Frequency in Lab

In [ ]:
raw.info['line_freq'] = 60 #The hertz of the powerlines fed into the lab 

## View Raw Data

Built-in plotting methods for Raw objects
https://mne.tools/stable/auto_tutorials/raw/40_visualize_raw.html

## Create BIDS Path

Importing data from EEG devices
https://mne.tools/stable/auto_tutorials/io/20_reading_eeg_data.html

In [ ]:
print('######################## Computing BIDS path. ########################')

In [ ]:
bids_path = mne_bids.BIDSPath(subject=subject,
                              session=session,
                              task=task,
                              root=bids_root) #An object that specifies the location of BIDS files

mne_bids.write_raw_bids(raw, bids_path=bids_path, events_data=events, format="BrainVision", allow_preload=True,
                        event_id=event_id, overwrite=True) #Writes the data

## Reports

### Raw Report

#### Set Path

In [ ]:
eeg_path = "eeg"
subname = "sub-"+subject
sesname = "ses-"+session
rawreports = "01_raw"

In [ ]:
#Makes EEG_OUTPUTS -> subject -> session -> eeg -> rawreports -> task
if not os.path.exists(os.path.join(output_path, subname, sesname, 
                                   eeg_path, rawreports)):
    os.makedirs(os.path.join(output_path, subname, sesname, 
                             eeg_path, rawreports))

In [ ]:
report_path = os.path.join(root_path, "EGI_OUTPUTS\\sub-"+subject+"\\ses-"+session+"\\eeg\\"+rawreports)

#### Save Raw Report

In [ ]:
rawreport = subname+"_"+sesname+"_"+task+'_report_raw.html'

In [ ]:
report = mne.Report(title='Raw Report')
report.add_raw(raw=raw, title='Raw', psd=False, scalings=dict(eeg=100e-6))  # omit PSD plot
raw_report_path = os.path.join(report_path, rawreport)
report.save(raw_report_path, overwrite=True)

### Events Report

In [ ]:
sfreq = raw.info['sfreq']
eventreport = subname+"_"+sesname+"_"+task+'_report_events.html'

report = mne.Report(title='Events example')
report.add_events(events=events, title='Events from "events"', sfreq=sfreq)
event_path = os.path.join(report_path, eventreport)

report.save(event_path, overwrite=True)

### Full Report

In [ ]:
dataset_report = make_report(root=bids_root)
full_report = mne.Report(title='Full Report')
full_report.add_html(html=dataset_report, title='full_report.html')
report_path = os.path.join(output_path, 'full_report.html')

full_report.save(report_path, overwrite=True)